In [3]:
import weave
from dotenv import load_dotenv
import os

load_dotenv()
project = os.getenv('WANDB_PROJECT')
weave.init(project)

/Users/ram/miniforge3/envs/sitewiz/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Logged in as Weights & Biases user: sitewiz-ram.
View Weave data at https://wandb.ai/sitewiz/Agents/weave


In [ ]:
print("{} (43% o) : {}")

In [1]:
"{hi}".format(hi=["hello", "world"])    

"['hello', 'world']"

In [3]:
import os
table_name =os.environ.get('AWS_REIIGION') or 'us-west-2'
table_name

'us-west-2'

In [3]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def get_project_id(org_name, project_number, project_name):
    # Get token from environment variable
    token = os.getenv('GITHUB_TOKEN')
    if not token:
        raise ValueError("GITHUB_TOKEN environment variable is not set")

    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/vnd.github.v3+json"
    }
    
    # GraphQL query to get project ID
    query = """
    query($org: String!, $number: Int!) {
        organization(login: $org) {
            projectV2(number: $number) {
                id
                title
            }
        }
    }
    """
    
    variables = {
        "org": org_name,
        "number": project_number
    }
    
    data = {
        "query": query,
        "variables": variables
    }
    
    try:
        response = requests.post(
            "https://api.github.com/graphql",
            json=data,
            headers=headers
        )
        response.raise_for_status()  # Raise exception for bad status codes
        
        result = response.json()
        
        # Check for errors in the GraphQL response
        if 'errors' in result:
            raise Exception(f"GraphQL Error: {result['errors']}")
        
        # Verify the project name matches and return the ID
        project_data = result.get('data', {}).get('organization', {}).get('projectV2', {})
        if project_data.get('title') == project_name:
            return project_data.get('id')
        raise ValueError(f"Project with name '{project_name}' not found")
        
    except requests.exceptions.RequestException as e:
        raise Exception(f"API request failed: {str(e)}")

if __name__ == "__main__":
    try:
        project_id = get_project_id(
            org_name="SitewizAI",
            project_number=21,
            project_name="Evaluations"
        )
        print(f"Project ID: {project_id}")
    except Exception as e:
        print(f"Error: {str(e)}")


Project ID: PVT_kwDOCOLC8s4AyzdZ


In [ ]:
import boto3

def create_timestamp_index():
    dynamodb = boto3.client('dynamodb')
    table_name = 'EvaluationsTable'
    
    try:
        response = dynamodb.update_table(
            TableName=table_name,
            AttributeDefinitions=[
                {
                    'AttributeName': 'type',
                    'AttributeType': 'S'
                },
                {
                    'AttributeName': 'timestamp',
                    'AttributeType': 'N'
                },
            ],
            GlobalSecondaryIndexUpdates=[
                {
                    'Create': {
                        'IndexName': 'type-timestamp-index',
                        'KeySchema': [
                            {
                                'AttributeName': 'type',
                                'KeyType': 'HASH'
                            },
                            {
                                'AttributeName': 'timestamp',
                                'KeyType': 'RANGE'
                            },
                        ],
                        'Projection': {
                            'ProjectionType': 'ALL'
                        },
                        'ProvisionedThroughput': {
                            'ReadCapacityUnits': 5,
                            'WriteCapacityUnits': 5
                        }
                    }
                }
            ]
        )
        print("Index creation initiated:", response)
    except Exception as e:
        print(f"Error creating index: {e}")

create_timestamp_index()

Error creating index: An error occurred (ValidationException) when calling the UpdateTable operation: One or more parameter values were invalid: Neither ReadCapacityUnits nor WriteCapacityUnits can be specified for index: TimestampIndex when BillingMode is PAY_PER_REQUEST


In [6]:
import boto3
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Initialize DynamoDB resource
dynamodb = boto3.resource('dynamodb')

# Get the PromptsTable
prompts_table = dynamodb.Table('PromptsTable')

# Get the PromptRefsTable
prompt_refs_table = dynamodb.Table('PromptRefsTable')

def get_all_prompts():
    """Fetch all prompts from DynamoDB PromptsTable."""
    try:
        response = prompts_table.scan()
        prompts = response.get('Items', [])
        return prompts
    except Exception as e:
        print(f"Error fetching prompts: {e}")
        return []

def add_prompts_to_refs_table(prompts):
    """Add all prompts to the PromptRefsTable."""
    try:
        for prompt in prompts:
            prompt_ref = {
                'ref': prompt['ref'],
                'version': prompt.get('version', 'N/A'),
                'content': prompt['content']
            }
            prompt_refs_table.put_item(Item=prompt_ref)
        print("All prompts added to PromptRefsTable successfully.")
    except Exception as e:
        print(f"Error adding prompts to PromptRefsTable: {e}")

# Fetch all prompts
prompts = get_all_prompts()

# Add prompts to PromptRefsTable
add_prompts_to_refs_table(prompts)

# Display the prompts
for prompt in prompts:
    print(prompt)

Error fetching prompts: An error occurred (ResourceNotFoundException) when calling the Scan operation: Requested resource not found
All prompts added to PromptRefsTable successfully.


In [1]:
print("%hello")

%hello


In [2]:
f'{['hello']}'

"['hello']"

In [1]:
import json
import weave

# Initialize Weave client
client = weave.init("Agents")

# Fetch evaluation traces
calls = client.get_calls(
    filter={"op_names": ["weave:///sitewiz/Agents/op/Evaluation.predict_and_score:*"]},
    sort_by=[{"field": "started_at", "direction": "desc"}],
)

# Number of traces to extract
num_traces = min(10, len(calls))  # Ensure we don't exceed available traces

# Function to clean conversation data
def filterConversation(conversation):
    for message in conversation:
        message.pop("models_usage", None)
        message.pop("_class_name", None)
        message.pop("_bases", None)
    return conversation

# Extract input-output pairs
traces = []
for i in range(num_traces):
    try:
        call = calls[i]

        trace = {
            "input": call.inputs.get("example", "N/A"),
            "output": call.output.get("scores", {}).get("score", "N/A"),
        }
        traces.append(trace)
    except Exception as e:
        print(e)

# Save to JSON file
with open("traces.json", "w") as json_file:
    json.dump(traces, json_file, indent=4)

print(f"Saved {num_traces} traces to traces.json")


/Users/ram/miniforge3/envs/sitewiz/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Logged in as Weights & Biases user: sitewiz-ram.
View Weave data at https://wandb.ai/sitewiz/Agents/weave


TypeError: Object of type WeaveObject is not JSON serializable

In [34]:
call = calls[10]

In [35]:
call.inputs

WeaveDict({'self': ObjectRef(entity='sitewiz', project='Agents', name='Evaluation', _digest='kvT3vbFYRA4jvIYf5nCrSZYYrPPBT74WYoPZe2WfM6w', _extra=()), 'model': OpRef(entity='sitewiz', project='Agents', name='analyze_problem', _digest='ixyV5UXE8SNNiOcXZF8LXGPbxoKf1bvYtiEN9QN2Y7k', _extra=()), 'example': ObjectRef(entity='sitewiz', project='Agents', name='Dataset', _digest='FC17lgH1yef1TYQjyPlImfpk9FmbPduDxjkd44qHsYc', _extra=('attr', 'rows', 'id', 'FuLYEuHBXbOz8tbw1QtHDpMBY2CMYob7ZPtxv4SSFUU'))})

In [ ]:
import weave
client = weave.init("Agents")
calls = client.get_calls(
    filter={"op_names": ["weave:///sitewiz/Agents/op/Evaluation.predict_and_score:*"]},
    sort_by=[{"field":"started_at","direction":"desc"}],
)
num_traces = 10
for i in range(num_traces):
    
    

In [7]:
call = calls[10]

In [28]:
call.output['model_output']
call.inputs['example']
trace = {
    'input': call.inputs['example'],
    'output': call.output['scores']['score'],
}

In [12]:
call.inputs['example']

WeaveDict({'question': 'Find a unique insight relevant to the business by diving into the data and finding interesting and useful metrics, basing off of an existing OKR.', 'business_context': '"1. What are your main business objectives for the next 1-6 months?\n(Select all that apply):"\t"\n\nIncrease conversions, Improve user engagement, Engagement priorities: video views, scroll depth, clicks, map module engagement: https://rednavel.link/Gn7X7PTL \n\n2. What Objective Key Results (OKRs) are most important to you?\n(Select up to three):"\t"\n\nDecrease bounce rate, Enhance click-through rates (CTR), Form fills is by far #1...but when you say "CTR" I want to clarity that is on-site and not via Ads or SEO. How many click a button / viewed a section with that button...we\'ve setup our Measurement this way via "element visibility" rules through GTM on the site and just wanted to clarity CTR means something different to us vs most. Viewing service pages and scroll + time on page (those tog

In [6]:
import boto3
import litellm
from litellm.utils import trim_messages
from litellm import completion
import json
# Flag to check if Vertex AI is initialized
vertex_ai_initialized = False

def get_api_key(secret_name):
    region_name = "us-east-1"
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )
    get_secret_value_response = client.get_secret_value(
        SecretId=secret_name
    )
    return json.loads(get_secret_value_response["SecretString"])

def initialize_vertex_ai():
    """Initialize Vertex AI with service account credentials"""
    global vertex_ai_initialized
    if not vertex_ai_initialized:
        AI_KEYS = get_api_key("AI_KEYS")
        litellm.api_key = AI_KEYS["LLM_API_KEY"]
        litellm.api_base = "https://llms.sitewiz.ai"
        litellm.enable_json_schema_validation = True
        
        vertex_ai_initialized = True

def run_completion_with_fallback(messages=None, prompt=None, models=["video"], response_format=None, temperature=None):
    """
    Run completion with fallback to evaluate.
    """
    initialize_vertex_ai()

    if messages is None:
        if prompt is None:
            raise ValueError("Either messages or prompt should be provided.")
        else:
            messages = [{"role": "user", "content": prompt}]

    trimmed_messages = messages
    try:
        trimmed_messages = trim_messages(messages, model)
    except Exception as e:
        pass

    for model in models:
        try:
            if response_format is None:
                response = completion(model="litellm_proxy/"+model, messages=trimmed_messages, temperature=temperature)
                content = response.choices[0].message.content
                return content
            else:
                response = completion(model="litellm_proxy/"+model, messages=trimmed_messages, response_format=response_format, temperature=temperature)
                content = json.loads(response.choices[0].message.content)  
                if isinstance(response_format, BaseModel):
                    response_format.model_validate(content)

                return content
        except Exception as e:
            print(f"Failed to run completion with model {model}. Error: {str(e)}")
    return None


In [7]:
with open('./extracted_code.txt', 'r', encoding='utf-8') as file:
    code = file.read()

prompt = f"""I have the following documents:

Code:
```
{code}
```


How can the code be improved?"""


output = run_completion_with_fallback(prompt=prompt)
print(output)

Okay, this is a large codebase, so let's break down potential areas for improvement.  I'll focus on general good practices, maintainability, and potential performance bottlenecks.  I'll provide specific examples based on the provided files, but keep in mind this is a general overview, and more in-depth analysis would be required for targeted optimization.

**1. General Code Structure & Design:**

*   **Modularity and Abstraction:** The code is broken into many files, which is good. However, look for opportunities to further abstract common functionalities into reusable classes or functions.  For example, database interaction logic or the logic for calling LLMs could be consolidated.
*   **Dependency Injection:** In many of the `create_*_agent` functions, there's a `get_llm_config` function passed as an argument. This is a good start, but consider using a dependency injection framework (even a simple one) to manage and provide these configurations more consistently and testably.  This w